In [3]:
%matplotlib inline
import time
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR


In [2]:
gpu_available = torch.cuda.is_available()

In [5]:
torch.optim.lr_scheduler()

TypeError: 'module' object is not callable

### Definition of some important parameters
Dataset to be used is defined here. According to the dataset, some other hyperparameters are set.

In [3]:
# This should, in the future, be set in CLI
chosen_dataset = 'CIFAR10'

datasets = {
    'MNIST': torchvision.datasets.MNIST,
    'CIFAR10': torchvision.datasets.CIFAR10
    #'FashionMNIST': torchvision.datasets.FashionMNIST
}

dataset = datasets[chosen_dataset]

possible_parameters = {
    'MNIST': {
        'ndf': 64,
        'ngf': 64,
        'code_size': 50,
        'n_channels': 1,
    },
    'CIFAR10': {
        'ndf': 64,
        'ngf': 64,
        'code_size': 100,
        'n_channels': 3,
    },
    #'FashionMNIST': {}
}

ngf = possible_parameters[chosen_dataset]['ngf']
ndf = possible_parameters[chosen_dataset]['ndf']
code_size = possible_parameters[chosen_dataset]['code_size']
n_channels = possible_parameters[chosen_dataset]['n_channels']

### Define data handling structures
- Batch size and number of child processes are to be set by user.
- Defines data transforms to be used in train and test phases
- Defines dataset and dataloader dictionaries

In [4]:
# These should, in the future, be set in CLI
batch_size = 16
number_processes = 4

data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(32),
        #transforms.RandomHorizontalFlip(),
        #transforms.ColorJitter(),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Scale(32),
        transforms.ToTensor(),
    ])
}

In [5]:
datasets = {
    'train': dataset('./datasets', train=True, download=False, transform=data_transforms['train']),
    'test': dataset('./datasets', train=False, download=False, transform=data_transforms['test'])
}

In [6]:
if gpu_available:
    n_train_samples = 1000
    n_test_samples = 100
    
else:
    n_train_samples = 500
    n_test_samples = 100

datasets['train'].train_data = datasets['train'].train_data[:n_train_samples]
datasets['train'].train_labels = datasets['train'].train_labels[:n_train_samples]

datasets['test'].test_data = datasets['test'].test_data[:n_test_samples]
datasets['test'].test_labels = datasets['test'].test_labels[:n_test_samples]

In [7]:
dataloaders = {
    'train': torch.utils.data.DataLoader(datasets['train'], batch_size=batch_size,
                                         shuffle=True, num_workers=number_processes),
    'test': torch.utils.data.DataLoader(datasets['train'], batch_size=batch_size,
                                        shuffle=False, num_workers=number_processes),
}
print('Length of training dataloader:', len(dataloaders['train']))
print('Length of test dataloader:', len(dataloaders['test']))

Length of training dataloader: 63
Length of test dataloader: 63


### Network definitions
Network hyperparameters are defined by the user or defined according to dataset choice.

In [8]:
class generator_network(nn.Module):
    # This model is sequential and will be implemented as such for simplicity
    def __init__(self, code_size, ngf, n_channels):
        super().__init__()
        self.code_size = code_size
        self.ngf = ngf
        self.n_channels = n_channels
        self.fc = nn.Sequential(
            nn.Linear(self.code_size, self.ngf*8 * 4 * 4),
            #nn.BatchNorm1d(self.ngf*8*4*4),
            nn.ReLU(),
        )
        self.deconv = nn.Sequential(
            # 4x4 -> 8x8
            nn.ConvTranspose2d(self.ngf*8, self.ngf*4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(self.ngf*4),
            nn.ReLU(),
            # 8x8 -> 16x16
            nn.ConvTranspose2d(self.ngf*4, self.ngf*2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(self.ngf*2),
            nn.ReLU(),
            # 16x16 -> 32x32
            nn.ConvTranspose2d(self.ngf*2, self.n_channels, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(self.n_channels),
            # If uncommented, 32x32 -> 64x64
            #nn.ReLU(),
            #nn.ConvTranspose2d(self.ngf, self.n_channels, kernel_size=4, stride=2, padding=1),
            #nn.BatchNorm2d(self.n_channels),
            nn.Tanh(),
        )
        
    def forward(self, input):
        input = input.view(-1, self.code_size)
        input = self.fc(input)
        input = input.view(-1, self.ngf*8, 4, 4)
        output = self.deconv(input)
        return output

In [9]:
class discriminator_network(nn.Module):
    def __init__(self, ndf, n_channels):
        super().__init__()
        self.ndf = ndf
        self.n_channels = n_channels
        self.conv = nn.Sequential(
            nn.Conv2d(self.n_channels, 96, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(96),
            nn.LeakyReLU(),
            nn.Conv2d(96, 96, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(96),
            nn.LeakyReLU(),
            nn.Conv2d(96, 96, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(96),
            nn.LeakyReLU(),
            nn.Dropout(),
            nn.Conv2d(96, 192, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(192),
            nn.LeakyReLU(),
            nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(192),
            nn.LeakyReLU(),
            nn.Conv2d(192, 192, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(192),
            nn.LeakyReLU(),
            nn.Dropout(),
            nn.Conv2d(192, 192, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(192),
            nn.LeakyReLU(),
        )
        self.n_conv_features = self._get_conv_output((3,32,32))
        self.fc = nn.Sequential(
            nn.Linear(self.n_conv_features, 192),
            nn.Linear(192, 100),
            nn.Linear(100, 10)
        )
        
    def forward(self, input):
        x = self.conv(input)
        x = x.view(-1, self.n_conv_features)
        output = self.fc(x).squeeze()
        return output
    
    def _get_conv_output(self, shape):
        bs = 1
        input = Variable(torch.rand(bs, *shape))
        output_feat = self.conv(input)
        n_size = output_feat.data.view(bs, -1).size(1)
        return n_size

### Auxiliary function definitions
- weights_init initializes Conv layers and BatchNorm layers

In [10]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.05)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.05)
        m.bias.data.fill_(0)

### Network creation step

In [11]:
D = discriminator_network(ndf, n_channels)
G = generator_network(code_size, ngf, n_channels)

D.apply(weights_init)
G.apply(weights_init)

print(G, '\n', D)

generator_network (
  (fc): Sequential (
    (0): Linear (100 -> 8192)
    (1): ReLU ()
  )
  (deconv): Sequential (
    (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU ()
    (6): ConvTranspose2d(128, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True)
    (8): Tanh ()
  )
) 
 discriminator_network (
  (conv): Sequential (
    (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.01)
    (3): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
    (5): LeakyReLU (0.01)
    

### Defining optimization parameters
- Optimizer: Adam
- LR: user defined, default = 0.0003 (3e-4)
- LR decay: linear decay over the epochs, to a minimum of 0.0001 (1e-4)
- Loss function: BCELoss

In [12]:
input = torch.FloatTensor(batch_size, code_size, 32, 32)
label = torch.FloatTensor(batch_size)
fixed_noise = torch.FloatTensor(batch_size, code_size, 1, 1).normal_(0, 1)
noise = torch.FloatTensor(batch_size)
real_label = 1
fake_label = 0

criterion = nn.BCELoss()
optimizerD = torch.optim.Adam(D.parameters(), lr=3e-4)
optimizerG = torch.optim.Adam(G.parameters(), lr=3e-4)

lin_decay = lambda epoch: np.clip(3 - epoch / 400, 1e-4, None)
D_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizerD, lin_decay)
G_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizerG, lin_decay)

if gpu_available:
    D.cuda()
    G.cuda()
    criterion.cuda()
    input, label = input.cuda(), label.cuda()
    fixed_noise, noise = fixed_noise.cuda(), noise.cuda()
    
input, label = Variable(input), Variable(label)
fixed_noise, noise = Variable(fixed_noise), Variable(noise)

### Training function

In [13]:
 def save_images(netG, noise, outputDir,epoch):
    # the first 64 samples from the mini-batch are saved.
    fake,_ = netG(fixed_noise)
    vutils.save_image(fake.data[0:64,:,:,:],
            '%s/fake_samples_epoch_%03d.png' % (outputDir, epoch), nrow=8)

def save_models(netG, netD, outputDir, epoch):
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outputDir, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outputDir, epoch))


def train_both_networks(num_epochs, dataloader, netD, netG, d_labelSmooth, outputDir,
                        model_option =1,binary = False, epoch_interval = 1):
    
    for epoch in range(num_epochs):
        for i, data in enumerate(dataloader, 0):
            start_iter = time.time()
            
            
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            # 1A - Train the detective network in the Real Dataset
            ###########################
            # train with real
            netD.zero_grad()
            real_cpu, _ = data
            batch_size = real_cpu.size(0)
            input.data.resize_(real_cpu.size()).copy_(real_cpu)
            label.data.resize_(batch_size).fill_(real_label - d_labelSmooth) # use smooth label for discriminator

            output = netD(input)
            errD_real = criterion(output, label)
            errD_real.backward()
            
            #######################################################
           
            #######################################################
            # 1B - Train the detective network in the False Dataset
            #######################################################
            
            D_x = output.data.mean()
            # train with fake
            noise.data.resize_(batch_size, code_size, 1, 1)
            if binary:
                bernoulli_prob.resize_(noise.data.size())
                noise.data.copy_(2*(torch.bernoulli(bernoulli_prob)-0.5))
            else:
                noise.data.normal_(0, 1)
            fake = netG(noise)
            label.data.fill_(fake_label)
            output = netD(fake.detach()) # add ".detach()" to avoid backprop through G
            errD_fake = criterion(output, label)
            errD_fake.backward() # gradients for fake/real will be accumulated
            D_G_z1 = output.data.mean()
            errD = errD_real + errD_fake
            optimizerD.step() # .step() can be called once the gradients are computed
            D_scheduler.step()

            #######################################################

            #######################################################
            # (2) Update G network: maximize log(D(G(z)))
            #  Train the faker with de output from the Detective (but don't train the Detective)
            #############3#########################################
            netG.zero_grad()
            label.data.fill_(real_label) # fake labels are real for generator cost
            output = netD(fake)
            errG = criterion(output, label)
            errG.backward(retain_graph=True) # True if backward through the graph for the second time()
            D_G_z2 = output.data.mean()
            optimizerG.step()
            G_scheduler.step()

            end_iter = time.time()
            
            #Print the info
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f Elapsed %.2f s'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2, end_iter-start_iter))
            
            #Save a grid with the pictures from the dataset, up until 64
            if i % 100 == 0:
                # the first 64 samples from the mini-batch are saved.
                vutils.save_image(real_cpu[0:64,:,:,:],
                        '%s/real_samples.png' % outputDir, nrow=8)
                fake = netG(fixed_noise)
                vutils.save_image(fake.data[0:64,:,:,:],
                        '%s/fake_samples_epoch_%03d.png' % (outputDir, epoch), nrow=8)
        if epoch % epoch_interval == 0:
            # do checkpointing
            torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outputDir, epoch))
            torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outputDir, epoch))

In [14]:
train_both_networks(num_epochs=25, dataloader=dataloaders['train'], netD=D, netG=G, d_labelSmooth=0.1,
                    outputDir='./generated_images', model_option=1, binary = False, epoch_interval=1)

[0/25][0/63] Loss_D: nan Loss_G: nan D(x): 0.1470 D(G(z)): 0.1928 / -0.5591 Elapsed 1.19 s
[0/25][1/63] Loss_D: nan Loss_G: nan D(x): -0.5474 D(G(z)): -0.5591 / -1.7376 Elapsed 0.33 s
[0/25][2/63] Loss_D: nan Loss_G: nan D(x): -1.7890 D(G(z)): -1.7362 / -2.9420 Elapsed 0.32 s
[0/25][3/63] Loss_D: nan Loss_G: nan D(x): -2.8662 D(G(z)): -3.0227 / -4.2261 Elapsed 0.30 s
[0/25][4/63] Loss_D: nan Loss_G: nan D(x): -3.9385 D(G(z)): -4.2271 / -5.2618 Elapsed 0.32 s
[0/25][5/63] Loss_D: nan Loss_G: nan D(x): -5.2030 D(G(z)): -5.1704 / -6.3691 Elapsed 0.32 s
[0/25][6/63] Loss_D: nan Loss_G: nan D(x): -6.3220 D(G(z)): -6.4502 / -7.6290 Elapsed 0.32 s
[0/25][7/63] Loss_D: nan Loss_G: nan D(x): -7.2405 D(G(z)): -7.5437 / -8.7960 Elapsed 0.35 s
[0/25][8/63] Loss_D: nan Loss_G: nan D(x): -8.3275 D(G(z)): -8.7876 / -9.8772 Elapsed 0.32 s
[0/25][9/63] Loss_D: nan Loss_G: nan D(x): -9.5952 D(G(z)): -10.0143 / -11.0778 Elapsed 0.32 s
[0/25][10/63] Loss_D: nan Loss_G: nan D(x): -10.8321 D(G(z)): -10.9150

[1/25][22/63] Loss_D: nan Loss_G: nan D(x): -147.0091 D(G(z)): -162.1294 / -164.9738 Elapsed 0.30 s
[1/25][23/63] Loss_D: nan Loss_G: nan D(x): -147.8991 D(G(z)): -167.0981 / -169.7894 Elapsed 0.32 s
[1/25][24/63] Loss_D: nan Loss_G: nan D(x): -150.1258 D(G(z)): -171.6676 / -174.7039 Elapsed 0.30 s
[1/25][25/63] Loss_D: nan Loss_G: nan D(x): -154.6290 D(G(z)): -173.5522 / -178.0585 Elapsed 0.30 s
[1/25][26/63] Loss_D: nan Loss_G: nan D(x): -158.4959 D(G(z)): -176.4958 / -179.9573 Elapsed 0.32 s
[1/25][27/63] Loss_D: nan Loss_G: nan D(x): -163.7581 D(G(z)): -181.6717 / -184.4836 Elapsed 0.32 s
[1/25][28/63] Loss_D: nan Loss_G: nan D(x): -165.5700 D(G(z)): -185.1011 / -191.1291 Elapsed 0.33 s
[1/25][29/63] Loss_D: nan Loss_G: nan D(x): -168.2978 D(G(z)): -189.9021 / -193.7545 Elapsed 0.32 s
[1/25][30/63] Loss_D: nan Loss_G: nan D(x): -172.5147 D(G(z)): -194.8741 / -199.3433 Elapsed 0.32 s
[1/25][31/63] Loss_D: nan Loss_G: nan D(x): -174.7050 D(G(z)): -199.8458 / -202.3457 Elapsed 0.32 s


[2/25][42/63] Loss_D: nan Loss_G: nan D(x): -643.7786 D(G(z)): -764.4562 / -774.8572 Elapsed 0.32 s
[2/25][43/63] Loss_D: nan Loss_G: nan D(x): -642.4745 D(G(z)): -769.8682 / -790.9824 Elapsed 0.30 s
[2/25][44/63] Loss_D: nan Loss_G: nan D(x): -661.8277 D(G(z)): -778.4151 / -798.8011 Elapsed 0.30 s
[2/25][45/63] Loss_D: nan Loss_G: nan D(x): -664.7339 D(G(z)): -805.3428 / -812.1306 Elapsed 0.32 s
[2/25][46/63] Loss_D: nan Loss_G: nan D(x): -677.9161 D(G(z)): -806.9878 / -817.5206 Elapsed 0.30 s
[2/25][47/63] Loss_D: nan Loss_G: nan D(x): -682.8476 D(G(z)): -819.3083 / -828.3215 Elapsed 0.30 s
[2/25][48/63] Loss_D: nan Loss_G: nan D(x): -703.7076 D(G(z)): -831.2719 / -839.0585 Elapsed 0.30 s
[2/25][49/63] Loss_D: nan Loss_G: nan D(x): -713.7194 D(G(z)): -835.5396 / -854.3846 Elapsed 0.30 s
[2/25][50/63] Loss_D: nan Loss_G: nan D(x): -719.4264 D(G(z)): -857.7681 / -866.6225 Elapsed 0.31 s
[2/25][51/63] Loss_D: nan Loss_G: nan D(x): -732.3589 D(G(z)): -869.3672 / -879.6365 Elapsed 0.32 s


[3/25][60/63] Loss_D: nan Loss_G: nan D(x): -1707.8700 D(G(z)): -1965.5031 / -1976.5673 Elapsed 0.32 s
[3/25][61/63] Loss_D: nan Loss_G: nan D(x): -1714.4688 D(G(z)): -1994.1616 / -1997.6616 Elapsed 0.30 s
[3/25][62/63] Loss_D: nan Loss_G: nan D(x): -1718.2122 D(G(z)): -2004.5363 / -2015.8457 Elapsed 0.17 s
[4/25][0/63] Loss_D: nan Loss_G: nan D(x): -1720.1729 D(G(z)): -2024.7097 / -2040.7870 Elapsed 0.32 s
[4/25][1/63] Loss_D: nan Loss_G: nan D(x): -1740.6113 D(G(z)): -2046.5078 / -2067.8093 Elapsed 0.32 s
[4/25][2/63] Loss_D: nan Loss_G: nan D(x): -1765.4406 D(G(z)): -2064.3357 / -2078.1804 Elapsed 0.31 s
[4/25][3/63] Loss_D: nan Loss_G: nan D(x): -1763.7827 D(G(z)): -2088.7170 / -2101.4019 Elapsed 0.32 s
[4/25][4/63] Loss_D: nan Loss_G: nan D(x): -1807.6678 D(G(z)): -2090.2327 / -2118.4705 Elapsed 0.32 s
[4/25][5/63] Loss_D: nan Loss_G: nan D(x): -1813.2771 D(G(z)): -2124.8286 / -2131.7539 Elapsed 0.32 s
[4/25][6/63] Loss_D: nan Loss_G: nan D(x): -1830.6719 D(G(z)): -2132.5071 / -21

[5/25][14/63] Loss_D: nan Loss_G: nan D(x): -3232.7822 D(G(z)): -3735.6067 / -3769.3176 Elapsed 0.30 s
[5/25][15/63] Loss_D: nan Loss_G: nan D(x): -3286.4490 D(G(z)): -3767.2175 / -3806.0020 Elapsed 0.32 s
[5/25][16/63] Loss_D: nan Loss_G: nan D(x): -3283.0496 D(G(z)): -3789.4507 / -3814.8840 Elapsed 0.30 s
[5/25][17/63] Loss_D: nan Loss_G: nan D(x): -3298.3413 D(G(z)): -3824.3689 / -3843.3638 Elapsed 0.30 s
[5/25][18/63] Loss_D: nan Loss_G: nan D(x): -3341.2974 D(G(z)): -3852.7219 / -3866.1838 Elapsed 0.32 s
[5/25][19/63] Loss_D: nan Loss_G: nan D(x): -3380.6091 D(G(z)): -3864.9070 / -3891.1372 Elapsed 0.39 s
[5/25][20/63] Loss_D: nan Loss_G: nan D(x): -3372.6672 D(G(z)): -3868.3987 / -3925.1692 Elapsed 0.32 s
[5/25][21/63] Loss_D: nan Loss_G: nan D(x): -3438.1855 D(G(z)): -3922.5449 / -3923.6545 Elapsed 0.34 s
[5/25][22/63] Loss_D: nan Loss_G: nan D(x): -3422.2273 D(G(z)): -3951.0317 / -3977.9319 Elapsed 0.32 s
[5/25][23/63] Loss_D: nan Loss_G: nan D(x): -3459.8577 D(G(z)): -3966.764

...

[20/25][2/63] Loss_D: nan Loss_G: nan D(x): -27408.6113 D(G(z)): -31227.2363 / -31110.3379 Elapsed 0.30 s
[20/25][3/63] Loss_D: nan Loss_G: nan D(x): -27478.9492 D(G(z)): -31128.3535 / -31029.7266 Elapsed 0.30 s
[20/25][4/63] Loss_D: nan Loss_G: nan D(x): -27458.2227 D(G(z)): -31139.4727 / -31160.3086 Elapsed 0.32 s
[20/25][5/63] Loss_D: nan Loss_G: nan D(x): -27309.9141 D(G(z)): -31063.4160 / -31236.1895 Elapsed 0.30 s
[20/25][6/63] Loss_D: nan Loss_G: nan D(x): -27512.5820 D(G(z)): -31125.0762 / -31039.2695 Elapsed 0.32 s
[20/25][7/63] Loss_D: nan Loss_G: nan D(x): -27714.3789 D(G(z)): -31101.5352 / -31102.1523 Elapsed 0.32 s
[20/25][8/63] Loss_D: nan Loss_G: nan D(x): -27359.6016 D(G(z)): -31216.0195 / -31095.2051 Elapsed 0.30 s
[20/25][9/63] Loss_D: nan Loss_G: nan D(x): -27267.9746 D(G(z)): -31165.0117 / -30991.8145 Elapsed 0.32 s
[20/25][10/63] Loss_D: nan Loss_G: nan D(x): -27253.3320 D(G(z)): -31095.7812 / -31151.9160 Elapsed 0.30 s
[20/25][11/63] Loss_D: nan Loss_G: nan D(x): 

[21/25][16/63] Loss_D: nan Loss_G: nan D(x): -27476.3223 D(G(z)): -31132.3750 / -31155.9160 Elapsed 0.30 s
[21/25][17/63] Loss_D: nan Loss_G: nan D(x): -27373.4512 D(G(z)): -31215.3438 / -31111.7383 Elapsed 0.30 s
[21/25][18/63] Loss_D: nan Loss_G: nan D(x): -27435.6426 D(G(z)): -31161.5234 / -31077.8848 Elapsed 0.30 s
[21/25][19/63] Loss_D: nan Loss_G: nan D(x): -27531.3184 D(G(z)): -31148.3047 / -31215.4961 Elapsed 0.30 s
[21/25][20/63] Loss_D: nan Loss_G: nan D(x): -27457.8789 D(G(z)): -31197.5000 / -31158.3984 Elapsed 0.32 s
[21/25][21/63] Loss_D: nan Loss_G: nan D(x): -27172.0195 D(G(z)): -31128.5234 / -31123.6484 Elapsed 0.30 s
[21/25][22/63] Loss_D: nan Loss_G: nan D(x): -27301.6094 D(G(z)): -31127.2227 / -31054.3770 Elapsed 0.30 s
[21/25][23/63] Loss_D: nan Loss_G: nan D(x): -27345.1523 D(G(z)): -31115.6289 / -31203.3750 Elapsed 0.30 s
[21/25][24/63] Loss_D: nan Loss_G: nan D(x): -27298.9902 D(G(z)): -31110.2930 / -31105.7988 Elapsed 0.32 s
[21/25][25/63] Loss_D: nan Loss_G: na

[22/25][30/63] Loss_D: nan Loss_G: nan D(x): -27473.5605 D(G(z)): -31192.3301 / -31073.6152 Elapsed 0.30 s
[22/25][31/63] Loss_D: nan Loss_G: nan D(x): -27360.8066 D(G(z)): -31137.2617 / -31293.1855 Elapsed 0.32 s
[22/25][32/63] Loss_D: nan Loss_G: nan D(x): -27362.8984 D(G(z)): -31196.8867 / -31045.0039 Elapsed 0.30 s
[22/25][33/63] Loss_D: nan Loss_G: nan D(x): -27510.0742 D(G(z)): -31123.7969 / -31149.2812 Elapsed 0.34 s
[22/25][34/63] Loss_D: nan Loss_G: nan D(x): -27325.2695 D(G(z)): -31066.9805 / -31070.4512 Elapsed 0.32 s
[22/25][35/63] Loss_D: nan Loss_G: nan D(x): -27365.0117 D(G(z)): -31144.4277 / -31156.0312 Elapsed 0.32 s
[22/25][36/63] Loss_D: nan Loss_G: nan D(x): -27329.3359 D(G(z)): -31064.1641 / -31146.5625 Elapsed 0.32 s
[22/25][37/63] Loss_D: nan Loss_G: nan D(x): -27346.3398 D(G(z)): -31146.7480 / -31137.1895 Elapsed 0.32 s
[22/25][38/63] Loss_D: nan Loss_G: nan D(x): -27261.1387 D(G(z)): -31117.1406 / -31127.5898 Elapsed 0.32 s
[22/25][39/63] Loss_D: nan Loss_G: na

[23/25][44/63] Loss_D: nan Loss_G: nan D(x): -27240.4824 D(G(z)): -31052.5605 / -31218.2773 Elapsed 0.30 s
[23/25][45/63] Loss_D: nan Loss_G: nan D(x): -27427.3242 D(G(z)): -31158.7598 / -31094.3848 Elapsed 0.32 s
[23/25][46/63] Loss_D: nan Loss_G: nan D(x): -27486.7852 D(G(z)): -31161.2246 / -31177.2383 Elapsed 0.30 s
[23/25][47/63] Loss_D: nan Loss_G: nan D(x): -27377.7910 D(G(z)): -31134.0176 / -31119.7480 Elapsed 0.30 s
[23/25][48/63] Loss_D: nan Loss_G: nan D(x): -27398.6816 D(G(z)): -31097.8008 / -31141.5820 Elapsed 0.30 s
[23/25][49/63] Loss_D: nan Loss_G: nan D(x): -27439.4043 D(G(z)): -31204.9980 / -31230.0801 Elapsed 0.32 s
[23/25][50/63] Loss_D: nan Loss_G: nan D(x): -27344.3320 D(G(z)): -31131.2090 / -31098.7734 Elapsed 0.32 s
[23/25][51/63] Loss_D: nan Loss_G: nan D(x): -27177.4844 D(G(z)): -31222.6172 / -31206.6992 Elapsed 0.32 s
[23/25][52/63] Loss_D: nan Loss_G: nan D(x): -27453.3711 D(G(z)): -31074.6328 / -31078.7637 Elapsed 0.32 s
[23/25][53/63] Loss_D: nan Loss_G: na

[24/25][58/63] Loss_D: nan Loss_G: nan D(x): -27456.0879 D(G(z)): -31117.3262 / -31155.1953 Elapsed 0.32 s
[24/25][59/63] Loss_D: nan Loss_G: nan D(x): -27294.6875 D(G(z)): -31085.3379 / -31048.6094 Elapsed 0.31 s
[24/25][60/63] Loss_D: nan Loss_G: nan D(x): -27288.2129 D(G(z)): -31210.2070 / -31111.8145 Elapsed 0.30 s
[24/25][61/63] Loss_D: nan Loss_G: nan D(x): -27404.2871 D(G(z)): -31131.5312 / -31174.8691 Elapsed 0.31 s
[24/25][62/63] Loss_D: nan Loss_G: nan D(x): -27412.5898 D(G(z)): -31093.9277 / -31106.1445 Elapsed 0.17 s
